In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pycaret.anomaly import *

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.model_selection import train_test_split

import os
import sys
import datetime

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
CFG = CFG()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', CFG.NCOLS)
pd.set_option('display.max_rows', CFG.NROWS)

In [22]:
len(train[train['Class'] == 1]) / len(train[train['Class'] == 0])

0.10288282034203589

In [61]:
orig = pd.read_csv(os.path.join(CFG.RAW_DATA, 'Pulsar.csv'))
train = pd.read_csv(os.path.join(CFG.RAW_DATA, 'train.csv')).drop(columns='id')
test = pd.read_csv(os.path.join(CFG.RAW_DATA, 'test.csv')).drop(columns='id')

In [78]:
# idx = train.groupby('Class', group_keys=False).apply(lambda x: x.sample(frac=0.02)).index
# train = train.loc[idx]
scaler = StandardScaler()
#train = train.groupby("Class", group_keys=False).sample(frac=0.05).reset_index(drop=True)

X = train.drop('Class', axis=1)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y = train['Class']

In [79]:
anom = setup(data = X, 
             silent = True)

,Description,Value
0,session_id,3099
1,Original Data,"(117564, 8)"
2,Missing Values,False
3,Numeric Features,8
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(117564, 8)"
9,CPU Jobs,-1


In [47]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [80]:
anom_model = create_model(model = 'histogram', fraction = 0.10288)

In [81]:
results = assign_model(anom_model)
results

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve,Anomaly,Anomaly_Score
0,0.88,2.13,-0.41,-0.40,1.61,2.19,-1.90,-1.18,1,13.14
1,-0.97,-1.71,-0.06,0.06,-0.32,-0.22,-0.26,-0.52,0,7.68
2,0.06,-1.13,-0.11,-0.15,-0.35,-0.52,0.04,-0.10,0,5.01
3,0.38,-0.13,-0.53,-0.29,-0.35,-0.26,0.04,-0.29,0,4.12
4,0.92,1.80,-0.54,-0.38,-0.41,-0.75,2.10,2.69,0,9.16
...,...,...,...,...,...,...,...,...,...,...
117559,0.87,1.64,-0.50,-0.37,-0.30,0.08,-0.38,-0.62,0,5.29
117560,0.05,0.95,-0.29,-0.34,-0.39,-0.54,0.87,0.61,0,6.24
117561,0.34,0.54,-0.39,-0.33,-0.30,-0.32,-0.28,-0.47,0,4.12
117562,-0.22,0.04,-0.06,-0.23,-0.37,-0.45,0.37,0.06,0,4.13


In [82]:
scaler = MinMaxScaler()

probs = scaler.fit_transform(results['Anomaly_Score'].values.reshape(-1, 1))
print(max(probs), min(probs), np.unique(probs))

[1.] [0.] [0.         0.00937988 0.01409616 ... 0.99273842 0.99283258 1.        ]


In [83]:
results['Class'] = y
print(log_loss(results['Class'], probs))

0.16997016694258543


In [77]:
results['Anomaly_Score'].max()

26.11195808409326

In [84]:
plot_model(anom_model, plot = 'tsne')

IntProgress(value=0, description='Processing: ', max=5)

KeyboardInterrupt: 